In [ ]:
import os
from os import listdir
from os.path import isfile, join
import sys
from xml.etree import ElementTree
from xml.etree.ElementTree import Element, SubElement
from lxml import etree
import codecs
import cv2

In [ ]:
XML_EXT = '.xml'
ENCODE_METHOD = "UTF8"


class PascalVocReader:

    def __init__(self, file_path):
        # shapes type:
        # [labbel, [(x1,y1), (x2,y2), (x3,y3), (x4,y4)], color, color, difficult]
        self.shapes = []
        self.file_path = file_path
        self.verified = False
        try:
            self.parse_xml()
        except:
            pass

    def get_shapes(self):
        return self.shapes

    def add_shape(self, label, bnd_box, difficult):
        x_min = int(float(bnd_box.find('xmin').text))
        y_min = int(float(bnd_box.find('ymin').text))
        x_max = int(float(bnd_box.find('xmax').text))
        y_max = int(float(bnd_box.find('ymax').text))
        points = [(x_min, y_min), (x_max, y_min), (x_max, y_max), (x_min, y_max)]
        self.shapes.append((label, points, None, None, difficult))

    def parse_xml(self):
        assert self.file_path.endswith(XML_EXT), "Unsupported file format"
        parser = etree.XMLParser(encoding=ENCODE_METHOD)
        xml_tree = ElementTree.parse(self.file_path, parser=parser).getroot()
        filename = xml_tree.find('filename').text
        try:
            verified = xml_tree.attrib['verified']
            if verified == 'yes':
                self.verified = True
        except KeyError:
            self.verified = False

        for object_iter in xml_tree.findall('object'):
            bnd_box = object_iter.find("bndbox")
            label = object_iter.find('name').text
            # Add chris
            difficult = False
            if object_iter.find('difficult') is not None:
                difficult = bool(int(object_iter.find('difficult').text))
            self.add_shape(label, bnd_box, difficult)
        return True


In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX
color=(0,0,255)
annotations=r'C:\Users\ITU\Desktop\test\annotations'     #Annotations folder
images=r'C:\Users\ITU\Desktop\test\images'     #Images folder
results=r'C:\Users\ITU\Desktop\test\results'      #Folder where you want to store results

files=[f for f in listdir(annotations)if isfile(join(annotations,f))]

for f in files:
    xml_path=join(annotations,f)
    thickness=0
    resolution=f.split('_')[-1].split('.')[0]
    if resolution=="1000x":
        thickness=33
    elif resolution=="400x":
        thickness=14
    else:
        thickness=3
#     print(f)
    img_name=f.split('x')[0]+'x.png'
    img_path=join(images,img_name)
#     print(img_path)
    # print(xml_path)
    img=cv2.imread(img_path)
    t_voc_parse_reader = PascalVocReader(xml_path)
    shapes = t_voc_parse_reader.get_shapes()
    # print(len(shapes))
    for s in shapes:
        org=s[1][0]
        org=(org[0], org[1]-8)
        color=(0,0,255)
        if s[0]=="ring":
            color=(0,0,255)
        elif s[0]=="trophozoite":
            color=(255,0,0)
        elif s[0]=="schizont":
            color=(0,255,0)
        else:
            color=(0,0,0)
# you can add the label on each boundary box plotted.
#         img = cv2.putText(img, s[0], org, font, 
#                     1, color, 2, cv2.LINE_AA)
        cv2.rectangle(img,s[1][0],s[1][2],color,thickness)

    cv2.imwrite(join(results,img_name),img)
